In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import etl.py module

In [3]:
# add src to module search path 

import os
import sys

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

if os.name == "nt":
    path_separator = "\\"
else:
    path_separator = "/"
    
module_path = f"{parent_dir}{path_separator}src"
sys.path.append(module_path)

# import custom module
from etl import DataPreprocessing

# How to use the module

To demonstrate the features of the module, we perform a very easy ETL pipeline, where we extract raw data from a csv, remove duplicate rows, and store the tranformed dataset in a csv file.

In [4]:
# class instantiation
my_class = DataPreprocessing()

In [5]:
# load raw data from csv
file_path = f'..{path_separator}data{path_separator}raw{path_separator}interview_signup.csv'
df = my_class.load_data_from_csv(file_path, 
                                 encoding='utf-8', 
                                 sep=',',
                                 header=0,
                                 dtype={'original_product_name': str,
                                        'postcode': str,
                                        'bundesland': str,
                                        'total_bonus': 'float64',
                                        'order_date': str})
df.shape

(318345, 5)

In [6]:
# delete duplicate rows
df = my_class.remove_duplicate_rows()
df.shape

(318175, 5)

In [7]:
# store preprocessed csv in folder data/processed
file_path_processed = f"..{path_separator}data{path_separator}processed{path_separator}demo_etl_module_processed.csv"
my_class.save_data_to_csv(file_path_processed, index=False)